In [1]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=52531ac0c3bded3fa38a4e5dcaf4e9828457fc25d41902fb722bcd5d631dc783
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
import time
import jsonlines
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [6]:
# Load the model
model = SentenceTransformer('all-MPNet-base-v2')  # replace with your model

# Set the paths
# Set the paths
input_file = '/content/drive/My Drive/Colab Notebooks/Item1_wrds_html_text.jsonl'
output_file = '/content/drive/My Drive/Colab Notebooks/Item1_wrds_html_text_embeds_MPNET.jsonl'



In [7]:
# Open output file
with jsonlines.open(output_file, mode='w') as writer:
    # Open input file
    with jsonlines.open(input_file, mode='r') as reader:

        batch = []
        for obj in reader:
            batch.append(obj)
            if len(batch) >= 1000:
                start_time = time.time()

                # Process batch
                for item in batch:
                    content = item['text']
                    
                    # Check if content is not empty
                    if content.strip():
                        # Break text into sentences
                        sentences = sent_tokenize(content)

                        # Count number of words in content
                        n_words = len(content.split())

                        # Embed each sentence and average embeddings
                        embeddings = model.encode(sentences)
                        avg_embedding = np.mean(embeddings, axis=0).tolist()

                        # Write data to file
                        writer.write({'file_id': item['file_id'], 'embedding': avg_embedding, 'n_words': n_words})
                    else:
                        # Write data for empty content
                        writer.write({'file_id': item['file_id'], 'embedding': [], 'n_words': 0})
                
                end_time = time.time()

                print(f"Processed batch of {len(batch)} files")
                print(f"Time taken: {end_time - start_time} seconds\n")
                
                # Clear the batch
                batch = []

Processed batch of 1000 files
Time taken: 230.23972511291504 seconds

Processed batch of 1000 files
Time taken: 223.25698852539062 seconds

Processed batch of 1000 files
Time taken: 219.7401237487793 seconds

Processed batch of 1000 files
Time taken: 223.23744440078735 seconds

Processed batch of 1000 files
Time taken: 232.5551941394806 seconds

Processed batch of 1000 files
Time taken: 218.49232172966003 seconds

Processed batch of 1000 files
Time taken: 211.43230271339417 seconds

Processed batch of 1000 files
Time taken: 218.49153876304626 seconds

Processed batch of 1000 files
Time taken: 220.759370803833 seconds

Processed batch of 1000 files
Time taken: 218.73102688789368 seconds

Processed batch of 1000 files
Time taken: 218.20271611213684 seconds

Processed batch of 1000 files
Time taken: 222.11893773078918 seconds

Processed batch of 1000 files
Time taken: 210.90910863876343 seconds

Processed batch of 1000 files
Time taken: 219.37683057785034 seconds

Processed batch of 1000 

OSError: ignored